# Choropleth Map

In [ ]:
import dash
import dash_leaflet as dl
from jupyter_dash import JupyterDash
import random
import pandas as pd

In [ ]:
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

In [ ]:
import geopandas as gpd
import json
zipfile = "zip://data/cb_2018_us_state_20m.zip"
states = gpd.read_file(zipfile)

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

### Data Prep: Merging COVID data with states geography

In [ ]:
covid = pd.read_csv('data/10-19-2021.csv').astype({'FIPS': int})

In [ ]:
coviddf = states.astype({
    'STATEFP': int
}).merge(covid, how='left', left_on='STATEFP', right_on='FIPS')

In [ ]:
from dash_extensions.javascript import arrow_function, assign, Namespace
import dash.html as html
import dash

### Dynamic Styling via javascript
Special note about the `hideout` property. It is a generic storage area to pass attributes defined by the constructor (or modified by the callback) and accessible by the javascript functions via the `context`

#### New Component:
The `dl.Colorbar` component is created for dash-leaflet. There are continuous and categorical color bars. Here's the basic usage:

```
dl.Colorbar(children, id, className, classes, colorscale, height, imax, min, nTicks, opacity, position, style, tickDecimals, tickText, tickValues, tooltip, unit, width)
```
There is a helper functions in `dash_leaflet.express`, `categorical_colorbar` for making categorical colorbars.

In [54]:
classes = [0, 10000, 30000, 100000, 300000, 1000000, 30000000, 10000000]
ctg = ['0+', '10k+', '30k+', '100k+', '300k+', '1M+', '3M+', '10M+']
colorscale = [
    '#FFEDA0', '#FED976', '#FEB24C', '#FD8D3C', '#FC4E2A', '#E31A1C',
    '#BD0026', '#800026'
]
# Create colorbar.
colorbar = dlx.categorical_colorbar(categories=ctg,
                                    colorscale=colorscale,
                                    width=300,
                                    height=30,
                                    position="bottomleft")

#### Inline

In [55]:
style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.props.hideout;  // get props from hideout
    const value = feature.properties[colorProp];  // get value the determines the color
    for (let i = 0; i < classes.length; ++i) {
        if (value > classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    return style;
}""")

#### Using Namespace

Alternatively create your own namespace and include as a javascript asset

*from the file* `assets/choropleth_demo.js`
```javascript
window.choropleth_demo = Object.assign({}, window.choropleth_demo, {
    covid: {
        StyleHandler: function(feature, context) {
            const {
                classes,
                colorscale,
                style,
                colorProp
            } = context.props.hideout; // get props from hideout
            const value = feature.properties[colorProp]; // get value the determines the color
            for (let i = 0; i < classes.length; ++i) {
                if (value > classes[i]) {
                    style.fillColor = colorscale[i]; // set the fill color according to the class
                }
            }
            return style;
        }
    }
})
```

In [56]:
style_handle = Namespace('choropleth_demo', 'covid')('StyleHandler')

In [57]:
style = dict(weight=2,
             opacity=1,
             color='white',
             dashArray='3',
             fillOpacity=0.7)

# Create geojson.
geojson = dl.GeoJSON(
    data=json.loads(coviddf.to_json()),
    options=dict(style=ns('StyleHandler')),  # Here's the magic
    hideout=dict(colorscale=colorscale,
                 classes=classes,
                 style=style,
                 colorProp="Confirmed"),
    id="geojson")

# Create app.
app = JupyterDash()
app.layout = html.Div([
    dl.Map(
        children=[dl.TileLayer(), geojson, colorbar], center=[39, -98], zoom=4)
],
                      style={
                          'width': '1000px',
                          'height': '500px'
                      },
                      id="map")
app.run_server(mode='inline', port=random.choice(range(2000, 10000)))

In [ ]:
app.layout = dl.Map(dl.TileLayer( asd='xx'), style={'width': '1000px', 'height': '500px'})

In [ ]:
dl.Colorbar( asd='xx')